# <b style="color: white; background-color: #00bbff; padding: 5px 10px; border-radius: 5px;">LIBRARY and SETTINGS</b>

In [1]:
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, when
import logging
# Carrega variáveis de ambiente
load_dotenv()
s3_endpoint = os.getenv("S3_ENDPOINT")
s3_access_key = os.getenv("S3_ACCESS_KEY")
s3_secret_key = os.getenv("S3_SECRET_KEY")

# -------------------------
# Configuração do Logging
# -------------------------
def setup_logger():
    # Cria o nome do arquivo de log com timestamp
    log_directory = "/opt/notebook/logs/"
    os.makedirs(log_directory, exist_ok=True)  # Garante que o diretório existe
    log_filename = f"gold_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    log_file = os.path.join(log_directory, log_filename)
    
    # Evita múltiplos handlers
    logger = logging.getLogger("minio_silver")
    if logger.handlers:  # Remove handlers existentes
        logger.handlers = []
    
    logger.setLevel(logging.INFO)
    
    # Formato do log
    formatter = logging.Formatter(fmt='{"level": "%(levelname)s", "message": "%(message)s"}')
    
    # Handler para console
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    
    # Handler para arquivo
    file_handler = logging.FileHandler(log_file)
    file_handler.setFormatter(formatter)
    
    # Adiciona ambos handlers
    logger.addHandler(console_handler)
    logger.addHandler(file_handler)
    
    return logger

logger = setup_logger()

def optimize_iceberg_table(spark, table_path):
    """Executa rotinas de otimização para tabela Iceberg com sintaxe compatível"""
    try:
        # 1. Compactação de arquivos de dados (sem comentários no SQL)
        spark.sql(f"""
            CALL local.system.rewrite_data_files(
                table => '{table_path}',
                options => map(
                    'target-file-size-bytes', '67108864',
                    'min-input-files', '5',
                    'min-file-size-bytes', '33554432'
                )
            )
        """)
        logger.info(f"✅ Compactação concluída para {table_path}")

        # 2. Expurgo de snapshots antigos
        retention_days = 7
        cutoff_date = (datetime.now() - timedelta(days=retention_days)).strftime('%Y-%m-%d %H:%M:%S')
        spark.sql(f"""
            CALL local.system.expire_snapshots(
                table => '{table_path}',
                older_than => timestamp '{cutoff_date}',
                retain_last => 3
            )
        """)
        logger.info(f"🗑️ Snapshots antigos removidos para {table_path}")

        # 3. Limpeza de arquivos órfãos
        spark.sql(f"""
            CALL local.system.remove_orphan_files(
                table => '{table_path}',
                older_than => timestamp '{cutoff_date}'
            )
        """)
        logger.info(f"🧹 Arquivos órfãos removidos para {table_path}")

        # 4. Otimização de metadados
        spark.sql(f"CALL local.system.rewrite_manifests('{table_path}')")
        logger.info(f"📦 Manifestos otimizados para {table_path}")

    except Exception as e:
        logger.error(f"❌ Falha na otimização de {table_path}: {str(e)}")
        raise   


# -------------------------
# Configuração do Spark com Otimizações
# -------------------------
def create_spark_session():
    """Cria uma SparkSession otimizada para Iceberg com MinIO"""
    from pyspark import SparkConf
    
    # Configuração inicial para controle de logs
    conf = SparkConf()
    conf.set("spark.logConf", "false")
    conf.set("spark.ui.showConsoleProgress", "false")
    conf.set("spark.driver.extraJavaOptions", "-Dlog4j.configuration=file:/opt/spark/conf/log4j.properties")
    
    # Verifica se os JARs existem
    iceberg_jar = "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.6.0.jar"
    required_jars = [
        iceberg_jar,
        "/opt/spark/jars/hadoop-aws-3.3.4.jar"
    ]
    
    # Configura a SparkSession
    spark = SparkSession.builder \
        .config(conf=conf) \
        .appName("IcebergOptimizedPipeline") \
        .config("spark.jars", ",".join([j for j in required_jars if os.path.exists(j)])) \
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
        .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
        .config("spark.sql.catalog.local.type", "hadoop") \
        .config("spark.sql.catalog.local.warehouse", "s3a://datalake/iceberg") \
        .config("spark.hadoop.fs.s3a.endpoint", s3_endpoint) \
        .config("spark.hadoop.fs.s3a.access.key", s3_access_key) \
        .config("spark.hadoop.fs.s3a.secret.key", s3_secret_key) \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.fast.upload", "true") \
        .config("spark.hadoop.fs.s3a.fast.upload.buffer", "disk") \
        .config("spark.hadoop.fs.s3a.connection.maximum", "100") \
        .config("spark.hadoop.fs.s3a.threads.max", "20") \
        .config("spark.sql.catalog.local.default-namespace", "default") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.shuffle.partitions", "4") \
        .config("spark.default.parallelism", "4") \
        .config("spark.sql.iceberg.handle-timestamp-without-timezone", "true") \
        .getOrCreate()
    
    # Configuração adicional de logs
    spark.sparkContext.setLogLevel("ERROR")
    return spark        


# <b style="color: white; background-color: #00bbff; padding: 5px 10px; border-radius: 5px;">FUNCTIONS AND EXECUTION</b>

In [2]:

# -------------------------
# Processamento de Clientes
# -------------------------
def process_clientes(spark):
    logger.info("🔧 Processando tabela de clientes para camada gold")
    
    try:
        clientes_df = spark.table("local.silver.clientes")
        
        clientes_gold = clientes_df.select(
            col("id").alias("cliente_id"),
            col("nome").alias("nome_cliente"),
            col("email"),
            col("data_cadastro"),
            when(col("status") == "ativo", 1).otherwise(0).alias("ativo"),
            year(col("data_cadastro")).alias("ano_cadastro"),
            month(col("data_cadastro")).alias("mes_cadastro"),
            col("created_at")
        )

        spark.sql("""
            CREATE TABLE IF NOT EXISTS local.gold.dim_clientes (
                cliente_id string,
                nome_cliente string,
                email string,
                data_cadastro string,
                ativo int,
                ano_cadastro int,
                mes_cadastro int,
                created_at timestamp
            )
            USING iceberg
            PARTITIONED BY (ano_cadastro, mes_cadastro)
            TBLPROPERTIES (
                'write.format.default' = 'parquet',
                'write.parquet.compression-codec' = 'snappy',
                'commit.retry.num-retries' = '10'
            )
        """)

        clientes_gold.writeTo("local.gold.dim_clientes").append()
        logger.info("✅ Tabela gold de clientes processada com sucesso!")
        
        table_path = 'local.gold.dim_clientes'
        # Otimização pós-escrita
        optimize_iceberg_table(spark, table_path)

    except Exception as e:
        logger.error(f"❌ Erro ao processar clientes: {str(e)}")
        raise

# -------------------------
# Processamento de Vendas
# -------------------------
def process_vendas(spark):
    logger.info("🔧 Processando tabela de vendas para camada gold")
    
    try:
        vendas_df = spark.table("local.silver.vendas")
        
        vendas_gold = vendas_df.select(
            col("id"),
            col("cliente_id"),
            col("produto"),
            col("categoria"),
            col("quantidade").cast("int"),
            col("preco_unitario").cast("double"),
            col("total").cast("double"),
            col("data_venda"),
            col("status"),
            year(col("data_venda")).alias("ano_venda"),
            month(col("data_venda")).alias("mes_venda"),
            col("created_at")
        )

        spark.sql("""
            CREATE TABLE IF NOT EXISTS local.gold.fato_vendas (
                id string,
                cliente_id string,
                produto string,
                categoria string,
                quantidade int,
                preco_unitario double,
                total double,
                data_venda string,
                status string,
                ano_venda int,
                mes_venda int,
                created_at timestamp
            )
            USING iceberg
            PARTITIONED BY (ano_venda, mes_venda)
            TBLPROPERTIES (
                'write.format.default' = 'parquet',
                'write.parquet.compression-codec' = 'snappy',
                'commit.retry.num-retries' = '10'
            )
        """)

        vendas_gold.writeTo("local.gold.fato_vendas").append()
        logger.info("✅ Tabela gold de vendas processada com sucesso!")
        
        table_path = 'local.gold.fato_vendas'
        # Otimização pós-escrita
        optimize_iceberg_table(spark, table_path)
                

    except Exception as e:
        logger.error(f"❌ Erro ao processar vendas: {str(e)}")
        raise

# -------------------------
# Função Principal
# -------------------------
def main():
    spark = None
    try:
        spark = create_spark_session()
        spark.sparkContext.setLogLevel("ERROR")

        process_clientes(spark)
        process_vendas(spark)
        
        logger.info("🏁 Processamento base da camada gold concluído com sucesso!")

    except Exception as e:
        logger.error(f"❌ Erro geral: {str(e)}")
    finally:
        if spark:
            spark.stop()
            logger.info("🛑 Sessão Spark finalizada")

if __name__ == "__main__":
    main()

ERROR StatusLogger Reconfiguration failed: No configuration found for '25af5db5' at 'null' in 'null'
ERROR StatusLogger Reconfiguration failed: No configuration found for 'Default' at 'null' in 'null'
25/09/13 19:24:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
{"level": "INFO", "message": "🔧 Processando tabela de clientes para camada gold"}
{"level": "INFO", "message": "✅ Tabela gold de clientes processada com sucesso!"}
{"level": "INFO", "message": "✅ Compactação concluída para local.gold.dim_clientes"}
{"level": "INFO", "message": "🗑️ Snapshots antigos removidos para local.gold.dim_clientes"}
{"level": "INFO", "message": "🧹 Arquivos órfãos removidos para local.gold.dim_clientes"}
{"level": "INFO", "message": "📦 Manifestos otimizados para local.gold.dim_clientes"}
{"level": "